In [ ]:
!pip install tensorflow==2.10
!pip install tensorflow-datasets
!pip install numpy
!pip install pandas

import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=["label", "message"])

train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

print(train_df.head())
print(test_df.head())

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['message'])

X_train = tokenizer.texts_to_sequences(train_df['message'])
X_test = tokenizer.texts_to_sequences(test_df['message'])

max_sequence_length = 100
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_test = pad_sequences(X_test, maxlen=max_sequence_length)

y_train = train_df['label'].values
y_test = test_df['label'].values

model = keras.Sequential([
    keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length),
    keras.layers.LSTM(64),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

print(f"Training Accuracy: {model.history.history['accuracy'][-1]}")
print(f"Validation Accuracy: {model.history.history['val_accuracy'][-1]}")

def predict_message(pred_text):
    seq = tokenizer.texts_to_sequences([pred_text])
    padded_seq = pad_sequences(seq, maxlen=max_sequence_length)

    prediction_prob = model.predict(padded_seq)[0][0]

    print(f"Prediction Probability: {prediction_prob}")

    label = 'spam' if prediction_prob > 0.5 else 'ham'

    return [prediction_prob, label]

def test_predictions():
    test_messages = ["how are you doing today",
                     "sale today! to stop texts call 98912460324",
                     "i dont want to go. can we try it a different day? available sat",
                     "our new mobile video service is live. just install on your phone to start watching.",
                     "you have won £1000 cash! call to claim your prize.",
                     "i'll bring it tomorrow. don't forget the milk.",
                     "wow, is your arm alright. that happened to me one time too"
                    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

test_predictions()
